# Today's cryptocurrencies historical data on by Market Cap 

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pendulum
from sqlalchemy import create_engine
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
import os
import requests
import json
from config import api_key

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23]


#### Context : Finance

#### Goal: Our goal is to create tables showing today’s 10 best/most profitable cryptocurrencies to invest in. We will look at historical data for the previous year and create a database by hourly and daily price changes. 

#### METHOD: we aim to do this by extracting data based on the ranks of the cryptocurrency and also the daily and hourly change of the cryptocurrency. 

#### FINDING DATA: Following are the sources for finding and collecting our data. Sources: Cryptocurrencies: https://coinmarketcap.com/ 
CSV files: https://www.cryptodatadownload.com/data/binance/
APIs: https://financialmodelingprep.com/api/v3/ https://rest.coinapi.io/v1/

#### Data collection tools: Splinter APIs BeautifulSoup

#### DATA AND CLEANUP: 
##### Cleanup process: Pandas SqlAlchemy Database: PostgreSQL

##### TEAM MEMBERS Juan Castaneda Elif Evrim Polat Nichole Edet

<div><h3 style="color:green;">Web Scraping - Obtaining today's best cryptocurrencies liste at https://coinmarketcap.com </h3></div>

In [3]:
# Obtaining Tables at the next link using pandas
url = 'https://coinmarketcap.com/'

In [4]:
tables = pd.read_html(url)

In [5]:
df = tables[0]
df.head()

,Unnamed: 0,#,Name,Price,24h %,7d %,Market Cap,Volume(24h),Circulating Supply,Last 7 Days,Unnamed: 10
0,NaN,1.0,Bitcoin1BTCBuy,"$63,130.68",0.00%,11.53%,"$1,179,465,279,485","$77,454,026,2731,226,884 BTC","18,682,918 BTC",NaN,NaN
1,NaN,2.0,Ethereum2ETHBuy,"$2,477.25",8.04%,23.93%,"$286,055,797,614","$36,803,232,79514,856,474 ETH","115,473,021 ETH",NaN,NaN
2,NaN,3.0,Binance Coin3BNBBuy,$545.68,2.11%,40.18%,"$84,324,921,817","$7,267,048,68313,317,501 BNB","154,532,785 BNB",NaN,NaN
3,NaN,4.0,XRP4XRP,$1.76,0.91%,91.53%,"$80,105,353,011","$28,078,151,16115,914,806,334 XRP","45,404,028,640 XRP",NaN,NaN
4,NaN,5.0,Tether5USDTBuy,$1.00,0.13%,0.05%,"$46,542,420,202","$167,237,635,609167,050,758,859 USDT","46,490,412,194 USDT",NaN,NaN


In [6]:
# using only the tables that will match our PostgreSQL database schema
columns = ['#', 'Name', 'Price']
cryptos_df = df[columns]
cryptos_df.head()

,#,Name,Price
0,1.0,Bitcoin1BTCBuy,"$63,130.68"
1,2.0,Ethereum2ETHBuy,"$2,477.25"
2,3.0,Binance Coin3BNBBuy,$545.68
3,4.0,XRP4XRP,$1.76
4,5.0,Tether5USDTBuy,$1.00


In [7]:
# converting # column values to integer
cryptos_df['#'] = cryptos_df['#'].astype(float).map("{:.0f}".format).copy()
cryptos_df.rename(columns={'#':'id','Name':'coin_name','Price':'latest_price'}, inplace=True)
# obtaiting only the first 10 coins
cryptos_df = cryptos_df[:10]

In [8]:
cryptos_df['latest_price'] = cryptos_df['latest_price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
cryptos_df.head()

,id,coin_name,latest_price
0,1,Bitcoin1BTCBuy,63130.68
1,2,Ethereum2ETHBuy,2477.25
2,3,Binance Coin3BNBBuy,545.68
3,4,XRP4XRP,1.76
4,5,Tether5USDTBuy,1.00


In [9]:
# Splitting Name column into 2 different columns which will containt the coin name and coin symbol using a number as a delimiter
# and using n=1 to do this only once, in case the coin name contains a number already
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
columnsplit.head()

,0,1,2
0,Bitcoin,1,BTCBuy
1,Ethereum,2,ETHBuy
2,Binance Coin,3,BNBBuy
3,XRP,4,XRP
4,Tether,5,USDTBuy


###### Formatting table to match our postgreSQL table in our DB

In [10]:
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
# since coin_symbol column originally contains a recommendation to "Buy" if applicable on coinmarketcap, we will get rid of that
# columnsplit[1] = columnsplit[1].str.replace('Buy', '', regex=True) by ignoring index 1
cryptos_df = cryptos_df.assign(coin_name=columnsplit[0],coin_symbol=columnsplit[2].str.replace('Buy',''))

In [11]:
cryptos_df

,id,coin_name,latest_price,coin_symbol
0,1,Bitcoin,63130.68,BTC
1,2,Ethereum,2477.25,ETH
2,3,Binance Coin,545.68,BNB
3,4,XRP,1.76,XRP
4,5,Tether,1.00,USDT
5,6,Cardano,1.45,ADA
6,7,Polkadot,43.04,DOT
7,8,Uniswap,38.32,UNI
8,9,Litecoin,278.58,LTC
9,10,Chainlink,40.83,LINK


In [12]:
# rearranging columns
columns = cryptos_df.columns.to_list()

In [13]:
# new order
columns = columns[:2] + [columns[-1]] + [columns[-2]]
columns

['id', 'coin_name', 'coin_symbol', 'latest_price']

In [14]:
cryptos_df = cryptos_df[columns]
cryptos_df

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,63130.68
1,2,Ethereum,ETH,2477.25
2,3,Binance Coin,BNB,545.68
3,4,XRP,XRP,1.76
4,5,Tether,USDT,1.00
5,6,Cardano,ADA,1.45
6,7,Polkadot,DOT,43.04
7,8,Uniswap,UNI,38.32
8,9,Litecoin,LTC,278.58
9,10,Chainlink,LINK,40.83


<div><h3 style="color:green;">Dowloading the CSV files if available at https://www.cryptodatadownload.com/data/binance/</h3></div>

In [15]:
# Obtaining symbols in the dataframe and storing in a list
symbols = cryptos_df['coin_symbol'].to_list()
symbols

['BTC', 'ETH', 'BNB', 'XRP', 'USDT', 'ADA', 'DOT', 'UNI', 'LTC', 'LINK']

###### Web Scraping using beautiulsoup

In [16]:
# URL of page to be scraped
url = 'https://www.cryptodatadownload.com/data/binance/'
# instantiating the webdriver for Chrome!!!
browser.visit(url)
# Getting the webpage content
html = browser.html
# parsing our html plain text to a BS object
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [17]:
prhs = soup.find_all('p')

In [18]:
bnb_cryptos = prhs[3]
links = bnb_cryptos.find_all('a')
complete_links = ["https://www.cryptodatadownload.com" + link['href'] for link in links if "cdd" in link['href']]

In [19]:
complete_links

['https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_minute.c

In [20]:
len(complete_links)

51

In [21]:
from fs.osfs import OSFS
with OSFS("./Resources") as myfs:
    if(not myfs.exists("datasets")):
        myfs.makedir("datasets")
        print("Created datasets folder")

In [22]:
crypto_data_found = []
for symbol in (symbols):
    for index,link in enumerate(complete_links):
        if symbol in link:
            if symbol == "USDT":
                crypto_data_found.append((symbol,False))
            else:
                crypto_data_found.append((symbol,True))
            break
        if index==len(complete_links)-1:
            crypto_data_found.append((symbol,False))


In [23]:
crypto_data_found

[('BTC', True),
 ('ETH', True),
 ('BNB', True),
 ('XRP', True),
 ('USDT', False),
 ('ADA', True),
 ('DOT', False),
 ('UNI', False),
 ('LTC', True),
 ('LINK', True)]

##### CoinAPI setup and use

In [20]:
def get_historic_data(symbol,time_start,time_end,period="1DAY",limit=10000):
    def to_csv(path):
        pass
    url = f"https://rest.coinapi.io/v1/ohlcv/{symbol}/USD/history?period_id={period}&limit={limit}&time_start={time_start}&time_end={time_end}"
    headers = {"X-CoinAPI-Key" : api_key}
    response = requests.get(url, headers = headers)

    if(response.status_code == 429):
        # API responses exhausted
        return "Too many requests."
    historic_data = response.json()

    return historic_data

In [23]:
today = str(pendulum.now()).split(".")[0]
year_ago= str(pendulum.now().subtract(years=1)).split(".")[0]
print(today)
print(year_ago)
get_historic_data("DOT",year_ago,today,)

2021-04-15T12:38:14
2020-04-15T12:38:14


[{'time_period_start': '2020-08-18T00:00:00.0000000Z',
  'time_period_end': '2020-08-19T00:00:00.0000000Z',
  'time_open': '2020-08-18T17:02:07.9180000Z',
  'time_close': '2020-08-18T23:59:25.8850000Z',
  'price_open': 50.0,
  'price_high': 50.7,
  'price_low': 2.9,
  'price_close': 3.1099,
  'volume_traded': 3421890.05619089,
  'trades_count': 8241},
 {'time_period_start': '2020-08-19T00:00:00.0000000Z',
  'time_period_end': '2020-08-20T00:00:00.0000000Z',
  'time_open': '2020-08-19T00:00:10.6230000Z',
  'time_close': '2020-08-19T23:58:50.5120000Z',
  'price_open': 3.1099,
  'price_high': 3.16,
  'price_low': 2.71,
  'price_close': 2.9175,
  'volume_traded': 1440781.98041395,
  'trades_count': 7624},
 {'time_period_start': '2020-08-20T00:00:00.0000000Z',
  'time_period_end': '2020-08-21T00:00:00.0000000Z',
  'time_open': '2020-08-20T00:00:04.7500000Z',
  'time_close': '2020-08-20T23:59:50.5420000Z',
  'price_open': 2.9175,
  'price_high': 2.96,
  'price_low': 2.73,
  'price_close': 2.

##### Obaiting CSV files

In [24]:
csv_download_links = []
for symbol,found in crypto_data_found:
    if found:
        csv_download_links.append(f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_d.csv")
        csv_download_links.append(f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_1h.csv")


In [25]:
csv_download_links

['https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LINKUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LINKUSDT_1h.csv']

In [26]:
import urllib.request

for link in csv_download_links:
    try:
        urllib.request.urlretrieve(link, f"./Resources/datasets/{link.split('/')[-1]}")
        print(link,"Downloaded")
    except Exception as e:
        print("can't download",link.split('/')[-1] )
        print(e)

https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_LINKUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_LINKUSDT_1